# Determines bounding boxes for each sulcus

This notebook determines bounding box around a sulcus. It uses a supervised database, in which each sulcus has been manually labelled.

# Imports

In [1]:
import sys
import os
import json
import tempfile
import colorado as cld

The following line permits to import deep_folding even if this notebook is executed from the notebooks subfolder (and no install has been launched):

 /notebooks/use_transform.ipynb  
 /deep_folding/__init__.py

In [2]:
sys.path.append((os.path.abspath('../')))
import deep_folding
print((os.path.dirname(deep_folding.__file__)))

/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/deep_folding


# User-specific variables

In [3]:
sulcus = 'S.T.s.ter.asc.ant.'

In [4]:
side = 'L'

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [5]:
src_dir = os.path.join(os.getcwd(), '../data/source/supervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

src_dir = /host/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/source/supervised


The target directory tgt_dir is where the files will be saved

In [6]:
tgt_dir = os.path.join(os.getcwd(), '../data/target/bbox')
tgt_dir = os.path.abspath(tgt_dir)
print(("tgt_dir = " + tgt_dir))

tgt_dir = /host/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/bbox


In [7]:
ref_dir = os.path.join(os.getcwd(), '../data/reference/bbox')
ref_dir = os.path.abspath(ref_dir)
print(("ref_dir = " + ref_dir))

ref_dir = /host/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/reference/bbox


In [8]:
print((sys.argv))

['ipykernel_launcher', '--ip=127.0.0.1', '--stdin=9088', '--control=9086', '--hb=9085', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"7a894fc1-4e4d-43f0-ae3c-5851d369db89"', '--shell=9087', '--transport="tcp"', '--iopub=9089', '--f=/tmp/tmp-18359KoFq5dJF034H.json']


Gets the normlized SPM file to get voxel size inside the program

In [9]:
norm_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
norm_dir = os.path.abspath(norm_dir)
sub_dir = "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition"

# Illustration of main program uses

### Using external calls

In [10]:
!python ../deep_folding/anatomist_tools/mask.py --help

usage: bounding_box.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-t TGT_DIR]
                       [-u SULCUS] [-i SIDE] [-p PATH_TO_GRAPH]
                       [-n NB_SUBJECTS] [-v OUT_VOXEL_SIZE] [-k SKELETON_FILE]

Computes mask and bounding box around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/lnao/PClean/database_learnclean/all/
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the output
                        transformation files. Default is :
                        /neurospin/dico/data/deep_folding/test/bbox
  -u SULCUS, --sulcus SULCUS
                        Sulcus name around

### By using the main function call

In [11]:
from deep_folding.anatomist_tools import mask
print((mask.__file__))

/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/deep_folding/anatomist_tools/mask.py


In [12]:
args = "--help"
argv = args.split(' ')

In [13]:
mask.main(argv)

usage: bounding_box.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-t TGT_DIR]
                       [-u SULCUS] [-i SIDE] [-p PATH_TO_GRAPH]
                       [-n NB_SUBJECTS] [-v OUT_VOXEL_SIZE] [-k SKELETON_FILE]

Computes mask and bounding box around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/lnao/PClean/database_learnclean/all/
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the output
                        transformation files. Default is :
                        /neurospin/dico/data/deep_folding/test/bbox
  -u SULCUS, --sulcus SULCUS
                        Sulcus name around

### By using the API function call

In [14]:
mask.bounding_box(src_dir=src_dir,
                  tgt_dir=tgt_dir,
                  sulcus=sulcus,
                  side=side,
                  number_subjects=0)

(0, 0, <soma.aims.Volume_FLOAT at 0x7fec135235e8>)

# Test example

In [15]:
unsupervised_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
reference_dir = os.path.join(os.getcwd(), '../data/reference')
skeleton_file = os.path.join(unsupervised_dir,
                            'ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation',
                            'Lskeleton_100206.nii.gz')

In [16]:
_, _, vol_mask = mask.bounding_box(src_dir=src_dir,
                                    tgt_dir=tgt_dir,
                                    sulcus=sulcus,
                                    side=side,
                                    number_subjects=1,
                                    voxel_size=1,
                                    skeleton_file=skeleton_file)

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [ ]:
vol_mask.shape 

(193, 229, 193, 1)

In [ ]:
from soma import aims
import moving_averages as ma

temp_dir = tempfile.mkdtemp()
mask_filename = f"mask.nii.gz"
aims.write(vol_mask, mask_filename)
bucket_filename = f"mask.bck"
cmd = f"AimsFileConvert -c Bucket -t VOID -e 1 -i {mask_filename} -o {bucket_filename}"
os.system(cmd)

# Displays bucket file
bucket, bucket_raw, dxyz, rot, tr = ma.load_bucket(bucket_filename)
m = cld.buket_to_mesh(bucket)
cld.draw(m)


In [ ]:
fig = cld.new_figure()
m = cld.buket_to_mesh(bucket)
m_raw = cld.buket_to_mesh(bucket_raw)

In [ ]:
bucket_raw

array([[132., 169.,  61.],
       [142., 162.,  62.],
       [133., 168.,  62.],
       ...,
       [153., 158., 107.],
       [153., 159., 107.],
       [154., 159., 108.]])

In [ ]:
#cld.draw(m,fig,color=[0,255,0])
cld.draw_numpy_bucket(bucket_raw)

# Mask test with more than 1 subject

In [ ]:
src_dir = "/host/neurospin/dico/data/bv_databases/human/pclean/all"

_, _, vol_mask = mask.bounding_box(src_dir=src_dir,
                                   tgt_dir=tgt_dir,
                                   sulcus=sulcus,
                                   side=side,
                                   number_subjects=10,
                                   out_voxel_size=1,
                                   skeleton_file=skeleton_file)

{'subject': 'sujet01', 'side': 'L', 'dir': '/host/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': '%(subject)s/t1mri/t1/default_analysis/folds/3.3/base2018_manual/%(side)s%(subject)s*.arg'}
{'subject': 's12590', 'side': 'L', 'dir': '/host/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': '%(subject)s/t1mri/t1/default_analysis/folds/3.3/base2018_manual/%(side)s%(subject)s*.arg'}
{'subject': 'ammon', 'side': 'L', 'dir': '/host/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': '%(subject)s/t1mri/t1/default_analysis/folds/3.3/base2018_manual/%(side)s%(subject)s*.arg'}
{'subject': 'vayu', 'side': 'L', 'dir': '/host/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': '%(subject)s/t1mri/t1/default_analysis/folds/3.3/base2018_manual/%(side)s%(subject)s*.arg'}
{'subject': 'osiris', 'side': 'L', 'dir': '/host/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': '%(subject)s/t1mri/t1/default_analysis/folds/3.3/base2018_m

In [ ]:
temp_dir = tempfile.mkdtemp()
mask_filename = f"{temp_dir}/mask.nii.gz"
aims.write(vol_mask, mask_filename)
bucket_filename = f"{temp_dir}/mask.bck"
cmd = f"AimsFileConvert -c Bucket -t VOID -e 1 -i {mask_filename} -o {bucket_filename}"
os.system(cmd)

# Displays bucket file
bucket, bucket_raw, dxyz, rot, tr = ma.load_bucket(bucket_filename)
m = cld.buket_to_mesh(bucket)
cld.draw(m)

# Result analysis

Prints the list of files of the target directory

In [ ]:
import os
tgt_dir_side = os.path.join(tgt_dir, side)
print(('\n'.join(os.listdir(tgt_dir_side))))

S.T.s.ter.asc.ant._left.json


Expected output (we read the bounding_box file from the reference directory):

In [ ]:
ref_dir_side = os.path.join(ref_dir, side)
ref_file = os.listdir(ref_dir_side)[0]
print("ref_file = ", ref_file, '\n')
with open(os.path.join(ref_dir_side,ref_file), 'r') as f:
    data_ref = json.load(f)
    print((json.dumps(data_ref, sort_keys=True, indent=4)))
    box_ref = {k: data_ref[k] for k in ['bbmin_voxel', 'bbmax_voxel', 'bbmin_MNI152', 'bbmin_MNI152']}

ref_file =  S.T.s.ter.asc.ant._left.json 

{
    "bbmax_AIMS_Talairach": [
        60.54534580957505,
        79.8109225368388,
        -5.382205677639462
    ],
    "bbmax_voxel": [
        137,
        153,
        78
    ],
    "bbmin_AIMS_Talairach": [
        34.22425249281275,
        58.9042680902665,
        -53.46315080433615
    ],
    "bbmin_voxel": [
        112,
        129,
        33
    ],
    "date": "2021-04-16 09:58:40",
    "git_sha": "b78f71123166d5b0224cbce8f2b432603b801f7d",
    "is_git": true,
    "nb_subjects": 1,
    "repo_working_dir": "/home/jc225751/Program/deep_folding",
    "side": "L",
    "src_dir": [
        "/home/jc225751/Program/deep_folding/data/source/supervised"
    ],
    "sulcus": "S.T.s.ter.asc.ant._left",
    "tgt_dir": "/home/jc225751/Program/deep_folding/data/target/bounding_box",
    "timestamp": 1618559920.21693
}


Obtained output (we read the bounding_box file from the target directory):

In [ ]:
tgt_file = os.listdir(tgt_dir_side)[0]
print("tgt_file = ", tgt_file, '\n')
with open(os.path.join(tgt_dir_side,tgt_file), 'r') as f:
    data_target = json.load(f)
    print((json.dumps(data_target, sort_keys=True, indent=4)))
    box_target = {k: data_ref[k] for k in ('bbmin_voxel', 'bbmax_voxel', 'bbmin_MNI152', 'bbmin_MNI152')}

tgt_file =  S.T.s.ter.asc.ant._left.json 

{
    "bbmax_AIMS_MNI152": [
        165.44244461549314,
        168.99963329321318,
        110.9886178030984
    ],
    "bbmax_voxel": [
        165.0,
        169.0,
        111.0
    ],
    "bbmin_AIMS_MNI152": [
        129.86412144733617,
        142.43008449084704,
        53.91592684551324
    ],
    "bbmin_voxel": [
        130.0,
        142.0,
        54.0
    ],
    "date": "2021-10-06 18:15:27",
    "git_sha": "9b08508e67eff6f1fa481c17371f730a59fe73e3",
    "is_git": true,
    "nb_subjects": 10,
    "out_voxel_size": 1,
    "repo_working_dir": "/host/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding",
    "side": "L",
    "src_dir": [
        "/host/neurospin/dico/data/bv_databases/human/pclean/all"
    ],
    "sulcus": "S.T.s.ter.asc.ant._left",
    "tgt_dir": "/host/home/jc225751/Runs/18_test_visualization_marco/Program/deep_folding/data/target/bbox",
    "timestamp": 1633536927.2198741
}


KeyError: 'bbmin_MNI152'

In [ ]:
box_target == box_ref

In [ ]:
box_ref